In [77]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from time import sleep
import os
import sys
import urllib.request

In [341]:
df_relmovie = pd.read_csv('semi.csv')
df_actor = pd.read_csv('actor.csv',index_col=0)
df_cf_actor= pd.read_csv('기간별박스오피스_12_2_18_7.csv',skiprows=2)

In [84]:
df_relmovie.head()

Unnamed: 0 rank      mname director       provider        date country  \
0           0   1          명량      김한민      씨제이이앤엠(주)  2014-07-30      한국   
1           1   2   신과함께-죄와 벌      김용화  롯데쇼핑㈜롯데엔터테인먼트  2017-12-20      한국   
2           2   3        국제시장      윤제균      씨제이이앤엠(주)  2014-12-17      한국   
3           3   4         베테랑      류승완      씨제이이앤엠(주)  2015-08-05      한국   
4           4   5         도둑들      최동훈         (주)쇼박스  2012-07-25      한국   

   n_scrn         money     guest genre    rating  grade  \
0    1587  1.357480e+11  17613682    사극  15세이상관람가   8.47   
1    1912  1.157000e+11  14410931   판타지  12세이상관람가   7.85   
2     966  1.108280e+11  14245998   드라마  12세이상관람가   9.01   
3    1064  1.050250e+11  13395400    액션  15세이상관람가   9.01   
4    1072  9.366557e+10  12983330    액션  15세이상관람가   7.60   

                                      actor  
0                              최민식|류승룡|조진웅|  
1                  하정우|차태현|주지훈|김향기|김동욱|마동석|  
2                          황정민|김윤진|오달수|정진영|  
3                          황정민|유아인|유해진|오달수|  
4  김윤석|김혜수|이정재|전지현|임달화|김해숙|오달수|김수현|이신제|증국상|

In [43]:
#df_relmovie.head(100)
df_cf_actor.head(5)

순위        영화명         개봉일          매출액   매출액\n점유율        누적매출액   \
0  1.0         명량   2014.7.30  1.357532e+11       0.0  1.357532e+11   
1  2.0  신과함께-죄와 벌  2017.12.20  1.156942e+11       0.0  1.156942e+11   
2  3.0       국제시장  2014.12.17  1.109309e+11       0.0  1.109309e+11   
3  4.0        베테랑    2015.8.5  1.051682e+11       0.0  1.051682e+11   
4  5.0        아바타  2009.12.17  4.072735e+08       0.0  1.253043e+11   

         관객수       누적관객수    스크린수      상영횟수  대표국적  국적                     제작사   \
0  17615039.0  17615039.0  1587.0  188646.0    한국  한국               (주)빅스톤픽쳐스   
1  14410250.0  14410250.0  1912.0  213200.0    한국  한국  리얼라이즈픽쳐스(주),(주)덱스터스튜디오   
2  14261581.0  14261581.0  1044.0  212664.0    한국  한국     (주)제이케이필름,씨제이이앤엠(주)   
3  13414009.0  13414009.0  1115.0  199231.0    한국  한국         (주)외유내강,(주)필름케이   
4     36226.0  13338863.0    68.0    1952.0    미국  미국           이십세기폭스필름코퍼레이션   

                        배급사        등급          장르       감독   \
0                  씨제이이앤엠(주)  15세이상관람가       사극,액션      김한민   
1              롯데쇼핑㈜롯데엔터테인먼트  12세이상관람가     판타지,드라마      김용화   
2                  씨제이이앤엠(주)  12세이상관람가         드라마      윤제균   
3                  씨제이이앤엠(주)  15세이상관람가      액션,드라마      류승완   
4  주식회사 해리슨앤컴퍼니,이십세기폭스코리아(주)  12세이상관람가  SF,액션,어드벤처  제임스 카메론   

                                                 배우   
0  최민식,류승룡,조진웅,김명곤,진구,이정현,권율,노민우,김태훈,이승준,오타니 료헤이,...  
1  하정우,차태현,주지훈,김향기,마동석,김동욱,도경수,오달수,임원희,장광,정해균,김수안...  
2  황정민,김윤진,오달수,정진영,장영남,라미란,김슬기,정영기,맹세창,김동원,김현중,김지...  
3  황정민,유아인,유해진,오달수,장윤주,김시후,오대환,정웅인,정만식,송영창,진경,유인영...  
4                                 샘 워싱턴,조 샐다나,시고니 위버

In [46]:
#df_relmovie1=df_relmovie.drop(['제작사','수입사','영화유형','영화형태','서울관객수','서울매출액','영화구분'],axis=1)

In [59]:
df_relmovie1.head(1)

순번 영화명   감독        배급사         개봉일  국적  전국스크린수         전국매출액        전국관객수  \
0  1   명량  김한민  씨제이이앤엠(주)  2014-07-30  한국  1,587   135748398910  17,613,682    

   장르        등급  
0  사극  15세이상관람가

In [48]:
df_relmovie2 = df_relmovie1.rename(columns={'순번':'rank','영화명':'mname','감독':'director','배급사':'provider','개봉일':'date','국적':'country','전국스크린수':'n_scrn','전국매출액':'money','전국관객수':'guest','장르':'genre','등급':'rating'}, inplace=False)

In [49]:
df_relmovie2.head(4)

rank      mname director       provider        date country  n_scrn  \
0   1          명량      김한민      씨제이이앤엠(주)  2014-07-30      한국  1,587    
1   2   신과함께-죄와 벌      김용화  롯데쇼핑㈜롯데엔터테인먼트  2017-12-20      한국  1,912    
2   3        국제시장      윤제균      씨제이이앤엠(주)  2014-12-17      한국    966    
3   4       시간이탈자      류승완      씨제이이앤엠(주)  2015-08-05      한국  1,064    

          money        guest genre    rating  
0  135748398910  17,613,682     사극  15세이상관람가  
1  115699634137  14,410,931    판타지  12세이상관람가  
2  110828014630  14,245,998    드라마  12세이상관람가  
3  105024756250  13,395,400     액션  15세이상관람가

In [17]:
cnt=0
for i in df_relmovie2['n_scrn']:
    i = float(str(i).replace(',',''))
    df_relmovie2.set_value(cnt, "n_scrn", i)
    cnt=cnt+1

/Users/kangjinhee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [18]:
cnt=0
for i in df_relmovie2['guest']:
    i = int(str(i).replace(',',''))
    df_relmovie2.set_value(cnt, "guest", i)
    cnt=cnt+1

/Users/kangjinhee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [60]:
df_relmovie2.head(1)

rank mname director   provider        date country  n_scrn         money  \
0   1     명량      김한민  씨제이이앤엠(주)  2014-07-30      한국  1,587   135748398910   

         guest genre    rating grade actor  
0  17,613,682     사극  15세이상관람가

In [52]:
df_relmovie2['grade'] = ''
df_relmovie2['actor'] = ''

In [53]:
df_relmovie2.head(1)

rank mname director   provider        date country  n_scrn         money  \
0   1     명량      김한민  씨제이이앤엠(주)  2014-07-30      한국  1,587   135748398910   

         guest genre    rating grade actor  
0  17,613,682     사극  15세이상관람가

In [ ]:
client_id = "D4rJqpB5sQrGOQZ098Xk"
client_secret = "PhSvV07Pho"

encText = urllib.parse.quote('위플래쉬')
url = "https://openapi.naver.com/v1/search/movie.xml?yearfrom=2011&yearto=2018&display=100&query=" + encText # xml 결과

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read().decode('utf-8')
    soup2 = BeautifulSoup(response_body,"html.parser")
    print(soup2.prettify())

In [24]:
for it_title in soup2.find_all('item'):
    print((it_title.title).text.lstrip('<br>').rstrip('</br>'))

아이언맨 3


In [111]:
'안소니 루소|조 루소|'.replace("|",",").rstrip(',').strip()

'안소니 루소,조 루소'

In [254]:
client_id = "D4rJqpB5sQrGOQZ098Xk"
client_secret = "PhSvV07Pho"

userrating=[]
actor=[]
director=[]
title=[]

keyword=df_relmovie2['mname']
cnt = 0
for name_cnt in range(len(keyword)):
    encText = urllib.parse.quote(keyword[name_cnt])
    url = "https://openapi.naver.com/v1/search/movie.xml?yearfrom=2011&yearto=2018&display=100&query=" + encText # xml 결과

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read().decode('utf-8')
        soup2 = BeautifulSoup(response_body,"html.parser")
        sleep(0.05)        
        for dir_cnt in soup2.find_all('director'):
            director.append(dir_cnt.text.replace("|",",").rstrip(',').strip())
        for it_title in soup2.find_all('item'):
            title.append(it_title.title.text.lstrip('<br>').rstrip('</br>'))
            
        for usr_rat in soup2.findAll('userrating'): 
            userrating.append(usr_rat.text) 
        for act_name in soup2.findAll('actor'):
            actor.append(act_name.text)

        for dir_cor in range(len(director)):
            if (director[dir_cor] == df_relmovie2[keyword==keyword[name_cnt]]['director'].values[0].strip())  & \
               (title[dir_cor] == df_relmovie2[keyword==keyword[name_cnt]]['mname'].values[0].strip()) :
#             if keyword[name_cnt] == director[dir_cor]:
                df_relmovie2.set_value(name_cnt,'grade',userrating[dir_cor])
                df_relmovie2.set_value(name_cnt,'actor',actor[dir_cor])
    #print(name_cnt)
print("-----------complete------------")

IndentationError: expected an indented block (<ipython-input-254-871f9f5af6f2>, line 41)

In [29]:
df_relmovie2.head(5)

rank      mname director       provider        date country n_scrn  \
0   1          명량      김한민      씨제이이앤엠(주)  2014-07-30      한국   1587   
1   2   신과함께-죄와 벌      김용화  롯데쇼핑㈜롯데엔터테인먼트  2017-12-20      한국   1912   
2   3        국제시장      윤제균      씨제이이앤엠(주)  2014-12-17      한국    966   
3   4       시간이탈자      류승완      씨제이이앤엠(주)  2015-08-05      한국   1064   
4   5         도둑들      최동훈         (주)쇼박스  2012-07-25      한국   1072   

          money     guest genre    rating grade actor  
0  135748398910  17613682    사극  15세이상관람가              
1  115699634137  14410931   판타지  12세이상관람가              
2  110828014630  14245998   드라마  12세이상관람가              
3  105024756250  13395400    액션  15세이상관람가              
4   93665568500  12983330    액션  15세이상관람가

In [118]:
#새로운 엑셀로 저장
#df_relmovie2.to_excel('jini.xlsx',sheet_name='Sheet1')


In [64]:
df_relmovie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8510 entries, 0 to 8509
Data columns (total 14 columns):
Unnamed: 0    8510 non-null int64
rank          8510 non-null object
mname         8510 non-null object
director      8378 non-null object
provider      8509 non-null object
date          8510 non-null object
country       8510 non-null object
n_scrn        8510 non-null int64
money         8510 non-null float64
guest         8510 non-null int64
genre         8468 non-null object
rating        8509 non-null object
grade         1018 non-null float64
actor         1014 non-null object
dtypes: float64(2), int64(3), object(9)
memory usage: 930.9+ KB


In [98]:
df_relmovie

Unnamed: 0    rank                  mname      director  \
0              0      1                      명량           김한민   
1              1      2               신과함께-죄와 벌           김용화   
2              2      3                    국제시장           윤제균   
3              3      4                     베테랑           류승완   
4              4      5                     도둑들           최동훈   
5              5      6                 7번방의 선물           이환경   
6              6      7                      암살           최동훈   
7              7      8             광해, 왕이 된 남자           추창민   
8              8      9                   택시운전사            장훈   
9              9     10                     부산행           연상호   
10            10     11                     변호인           양우석   
11            11     12            어벤져스: 인피니티 워   안소니 루소,조 루소   
12            12     13        어벤져스: 에이지 오브 울트론         조스 웨던   
13            13     14                    겨울왕국   크리스 벅,제니퍼 리   
14            14     15                   인터스텔라      크리스토퍼 놀란   
15            15     16                    검사외전           이일형   
16            16     17                      관상           한재림   
17            17     18                  아이언맨 3         쉐인 블랙   
18            18     19                    설국열차           봉준호   
19            19     20           캡틴 아메리카: 시빌 워   안소니 루소,조 루소   
20            20     21                  수상한 그녀           황동혁   
21            21     22            해적: 바다로 간 산적           이석훈   
22            22     23                      공조           김성훈   
23            23     24                 트랜스포머 3        마이클 베이   
24            24     25                    히말라야           이석훈   
25            25     26          미션임파서블:고스트프로토콜        브래드 버드   
26            26     27                      밀정           김지운   
27            27     28                  최종병기 활           김한민   
28            28     29                      써니           강형철   
29            29     30             스파이더맨: 홈 커밍          존 와츠   
...          ...     ...                    ...           ...   
8480        8480  8,481              1944 워 히어로       자스민 디즈더   
8481        8481  8,482            2차 세계대전 타임트랙    에밀 에드윈 스미스   
8482        8482  8,483          20대 같은 40대 미시녀      긴다이치 코고로   
8483        8483  8,484   2025 지구전쟁: 얼리전스 오브 파워        마이클 크럼   
8484        8484  8,485             2035 매드 레이스  크리스토퍼 제임스 밀러   
8485        8485  8,486          2047: 지구 최후의 날      알렉산드로 카폰   
8486        8486  8,487          24시간 달리는 쾌락 버스         료 카미이   
8487        8487  8,488             28살 미망인의 속옷          긴 류지   
8488        8488  8,489          3데이즈: 라스트 서바이벌        길 벨로우스   
8489        8489  8,490              3분 파트너-무삭제           박주영   
8490        8490  8,491                 400 데이즈       매트 오스터만   
8491        8491  8,492            50가지 욕망의 그림자         자레드 콘   
8492        8492  8,493       50대 장모님의 불타는 사위사랑           김두민   
8493        8493  8,494              666 사탄의 아들           NaN   
8494        8494  8,495         AV스타: 과거를 지워주세요      케이치 후지와라   
8495        8495  8,496              AV제작사의 여직원        나베 카츠오   
8496        8496  8,497        AV 아이돌 일본제일전 도전기     토미오카 타다후미   
8497        8497  8,498        D컵 여형사-가슴으로 느껴져요           카즈야   
8498        8498  8,499               E-19 바이러스           NaN   
8499        8499  8,500         F.O.E.: 에프.오.이.       키오니 왁스먼   
8500        8500  8,501            H컵 이상 거유 여배우       다케다 히데오   
8501        8501  8,502              P.S. 걸 무삭제           원석호   
8502        8502  8,503                   S걸의 덫     그레고리 하타나카   
8503        8503  8,504                VIP전용 여자       카네다 사토시   
8504        8504  8,505             VJ의 은밀한 사생활        오카다 히로   
8505        8505  8,506                av감독의 아내       쿠도 마사노리   
8506        8506  8,507                     만화방           허재형   
8507        8507  8,508                       벡      츠츠미 유키히코   
8508        8508  8,509                     클래쉬         레 탄 선   
8509        8509  8,510                 트

In [361]:
df_relmovie['grade']=df_relmovie['grade'].replace(np.nan,-1)
df_cf_actor['배우']=df_cf_actor['배우'].replace(np.nan,'없음')
df_relmovie['actor']=df_relmovie['actor'].replace(np.nan,-1)


In [382]:
userrating=[]
tmp=[]
cnt=0

for i in range(len(df_relmovie['grade'])):
    
    if ((df_relmovie['grade'][i] == -1) or ( df_relmovie['actor'][i] == -1)):
        #print(df_relmovie['mname'][i])
        name=df_relmovie['mname'][i]
        encText = urllib.parse.quote(name)
        url = "https://openapi.naver.com/v1/search/movie.xml?yearfrom=2011&yearto=2018&display=100&query=" + encText # xml 결과
        
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if(rescode==200):
            response_body = response.read().decode('utf-8')
            soup2 = BeautifulSoup(response_body,"html.parser")
            sleep(0.05)        
            #print(soup2.prettify())
            
            for usr_rat in soup2.findAll('userrating'): 
                userrating.append(usr_rat.text)
                
            for act_cnt in soup2.findAll('actor'):
                
                if(len(act_cnt.text.replace("|",",").rstrip(',').strip().split(','))==0):
                    act_t= np.nan
                elif(len(act_cnt.text.replace("|",",").rstrip(',').strip().split(','))==1):
                    act_t= np.nan
                    act_f= act_cnt.text.replace("|",",").rstrip(',').strip().split(',')[0]
                else:
                    act_f= act_cnt.text.replace("|",",").rstrip(',').strip().split(',')[0]
                    act_t=act_cnt.text.replace("|",",").rstrip(',').strip().split(',')[1]
                print("---------------------------------------------------")
                
                
                if(len(df_cf_actor[df_cf_actor['영화명']==name]['배우'])==0):
                    print("none")
                else:
                    act_csv=[]
                    print(cnt)
                    print(name)
                    act_csv=df_cf_actor[df_cf_actor['영화명']==name]['배우'].values[0].split(',')
                    
                    if(act_f in act_csv[0:3]):                        
                        df_relmovie.set_value(i,'grade',userrating[cnt])
                        df_relmovie.set_value(i,'actor',str(act_cnt.text).strip('[,]').replace('\'','').replace(', ','|'))
                    elif(act_t in act_csv[0:3]):
                        df_relmovie.set_value(i,'grade',userrating[cnt])
                        df_relmovie.set_value(i,'actor',str(act_cnt.text).strip('[,]').replace('\'','').replace(', ','|'))

                cnt=cnt+1
                
                #print("-------")
                
                
                

---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
2
빅 히어로
---------------------------------------------------
3
빅 히어로
---------------------------------------------------
4
빅 히어로
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
----------------------

---------------------------------------------------
218
가문의 영광5 - 가문의 귀환
---------------------------------------------------
none


/Users/kangjinhee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/kangjinhee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


---------------------------------------------------
220
점박이:한반도의 공룡3D
---------------------------------------------------
221
지오스톰
---------------------------------------------------
none
---------------------------------------------------
223
맨 인 더 다크
---------------------------------------------------
none
---------------------------------------------------
225
슈퍼배드 2
---------------------------------------------------
226
썬더와 마법저택
---------------------------------------------------
227
헝거게임: 모킹제이
---------------------------------------------------
228
헝거게임: 모킹제이
---------------------------------------------------
none
---------------------------------------------------
230
헝거게임: 더 파이널
---------------------------------------------------
231
하늘에서 음식이 내린다면 2
---------------------------------------------------
232
토르: 마법망치의 전설
---------------------------------------------------
233
레지던트 이블: 파멸의 날
---------------------------------------------------
234
안녕, 헤이즐
---------------------------

/Users/kangjinhee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/kangjinhee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


...

---------------------------------------------------
2676
미니 자이언트
---------------------------------------------------
none
---------------------------------------------------
2678
슈퍼히어로
---------------------------------------------------
2679
슈퍼히어로
---------------------------------------------------
2680
슈퍼히어로
---------------------------------------------------
2681
슈퍼히어로
---------------------------------------------------
2682
슈퍼히어로
---------------------------------------------------
2683
슈퍼히어로
---------------------------------------------------
2684
슈퍼히어로
---------------------------------------------------
2685
슈퍼히어로
---------------------------------------------------
2686
슈퍼히어로
---------------------------------------------------
2687
슈퍼히어로
---------------------------------------------------
2688
슈퍼히어로
---------------------------------------------------
2689
슈퍼히어로
---------------------------------------------------
2690
슈퍼히어로
---------------------------------------------------
2691
슈퍼

---------------------------------------------------
2812
로보싸커
---------------------------------------------------
2813
극장판 프리즘스톤 올스타 셀렉션
---------------------------------------------------
none
---------------------------------------------------
2815
나의 사적인 여자친구
---------------------------------------------------
2816
드래곤 기사단
---------------------------------------------------
2817
드래곤 기사단
---------------------------------------------------
2818
드래곤 기사단
---------------------------------------------------
2819
프리 스테이트
---------------------------------------------------
2820
프리 스테이트
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
-------------

---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
2956
노트르담 드 파리
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------

---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
---------------------------------------------------
none
-------------------------------

KeyboardInterrupt: 

In [388]:
df_relmovie['actor']=df_relmovie['actor'].replace(-1,np.nan)

In [389]:
df_relmovie.head(1000)

Unnamed: 0    rank                                  mname  \
0             0      1                                      명량   
1             1      2                               신과함께-죄와 벌   
2             2      3                                    국제시장   
3             3      4                                     베테랑   
4             4      5                                     도둑들   
5             5      6                                 7번방의 선물   
6             6      7                                      암살   
7             7      8                             광해, 왕이 된 남자   
8             8      9                                   택시운전사   
9             9     10                                     부산행   
10           10     11                                     변호인   
11           11     12                            어벤져스: 인피니티 워   
12           12     13                        어벤져스: 에이지 오브 울트론   
13           13     14                                    겨울왕국   
14           14     15                                   인터스텔라   
15           15     16                                    검사외전   
16           16     17                                      관상   
17           17     18                                  아이언맨 3   
18           18     19                                    설국열차   
19           19     20                           캡틴 아메리카: 시빌 워   
20           20     21                                  수상한 그녀   
21           21     22                            해적: 바다로 간 산적   
22           22     23                                      공조   
23           23     24                                 트랜스포머 3   
24           24     25                                    히말라야   
25           25     26                          미션임파서블:고스트프로토콜   
26           26     27                                      밀정   
27           27     28                                  최종병기 활   
28           28     29                                      써니   
29           29     30                             스파이더맨: 홈 커밍   
..          ...     ...                                    ...   
970         970    971                                     얼리맨   
971         971    972                                 감바의 대모험   
972         972    973                 극장판 소드 아트 온라인 -오디널 스케일-   
973         973    974                        고스트 라이더 : 복수의 화신   
974         974    975                                 우먼 인 블랙   
975         975    976                        정무문 : 100대 1의 전설   
976         976    977                                     여교사   
977         977    978                                     존 윅   
978         978    979                      최강전사 미니특공대: 영웅의 탄생   
979         979    980                                 로맨틱 크라운   
980         980    981                                      터키   
981         981    982         극장판 도라에몽 : 진구의 우주영웅기~스페이스 히어로즈~   
982         982    983                                 미라클 벨리에   
983         983    984                  다이노소어 어드벤처 : 백악기 공룡대백과   
984         984    985                                 배우는 배우다   
985         985    986                               안녕, 나의 소녀   
986         986    987                                   셀프/리스   
987         987    988                                  스틸 앨리스   
988         988    989                                      직쏘   
989         989    990                                러브 앤 프렌즈   
990         990    991   극장판 파워레인저: 캡틴포스 VS 미라클포스 199 히어로 대결전    
991         991    992                                 피터와 드래곤   
992         992    993                                      히트   
993         993    994                                      당갈   
994         994    995                베이블레이드 버스트 갓 : 갓 발키리의 탄생   
995         995    996                              맨 프롬 UNCLE   
996         996    997                                  레드 스패로   
997         997    998                                 마지막 4중주   
998         998    999                       

In [395]:
df_realmovie[df_relmovie['mname']=='고스트 라이더 : 복수의 화신']

/Users/kangjinhee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Empty DataFrame
Columns: [Unnamed: 0, rank, mname, director, provider, date, country, n_scrn, money, guest, genre, rating, grade, actor]
Index: []

In [397]:
#df_realmovie = df_relmovie.dropna(axis=0)

In [398]:
df_realmovie.head(100)

Unnamed: 0  rank               mname       director  \
0             0    1                   명량            김한민   
1             1    2            신과함께-죄와 벌            김용화   
2             2    3                 국제시장            윤제균   
3             3    4                  베테랑            류승완   
4             4    5                  도둑들            최동훈   
5             5    6              7번방의 선물            이환경   
6             6    7                   암살            최동훈   
7             7    8          광해, 왕이 된 남자            추창민   
8             8    9                택시운전사             장훈   
9             9   10                  부산행            연상호   
10           10   11                  변호인            양우석   
11           11   12         어벤져스: 인피니티 워    안소니 루소,조 루소   
12           12   13     어벤져스: 에이지 오브 울트론          조스 웨던   
13           13   14                 겨울왕국    크리스 벅,제니퍼 리   
14           14   15                인터스텔라       크리스토퍼 놀란   
15           15   16                 검사외전            이일형   
16           16   17                   관상            한재림   
17           17   18               아이언맨 3          쉐인 블랙   
18           18   19                 설국열차            봉준호   
19           19   20        캡틴 아메리카: 시빌 워    안소니 루소,조 루소   
20           20   21               수상한 그녀            황동혁   
21           21   22         해적: 바다로 간 산적            이석훈   
22           22   23                   공조            김성훈   
23           23   24              트랜스포머 3         마이클 베이   
24           24   25                 히말라야            이석훈   
25           25   26       미션임파서블:고스트프로토콜         브래드 버드   
26           26   27                   밀정            김지운   
27           27   28               최종병기 활            김한민   
28           28   29                   써니            강형철   
29           29   30          스파이더맨: 홈 커밍           존 와츠   
..          ...   ...                 ...            ...   
71           71   72            토르: 라그나로크       타이카 와이티티   
72           72   73           어메이징 스파이더맨           마크 웹   
73           73   74      님아, 그 강을 건너지 마오            진모영   
75           75   76           군도: 민란의 시대            윤종빈   
76           76   77   범죄와의 전쟁: 나쁜놈들 전성시대            윤종빈   
77           77   78                 주토피아  바이론 하워드,리치 무어   
78           78   79           엣지 오브 투모로우         더그 라이만   
79           79   80                  신세계            박훈정   
80           80   81             신비한 동물사전        데이빗 예이츠   
81           81   82                  도가니            황동혁   
82           82   83           내 아내의 모든 것            민규동   
83           83   84                  판도라            박정우   
84           84   85                  연가시            박정우   
85           85   86                  강철비            양우석   
86           86   87        해리포터와 죽음의 성물2        데이빗 예이츠   
87           87   88   엑스맨: 데이즈 오브 퓨처 패스트        브라이언 싱어   
88           88   89                  아가씨            박찬욱   
89           89   90         어메이징 스파이더맨 2           마크 웹   
90           90   91                  용의자            원신연   
91           91   92                건축학개론            이용주   
92           92   93                  댄싱퀸            이석훈   
93           93   94                    꾼            장창원   
94           94   95         혹성탈출: 반격의 서막          맷 리브스   
95           95   96              타짜-신의 손            강형철   
96           96   97                쿵푸팬더3  여인영,알레산드로 칼로니   
97           97   98       캡틴 아메리카: 윈터 솔져    조 루소,안소니 루소   
98           98   99                 박수건달            조진규   
99           99  100     조선명탐정 : 사라진 놉의 딸            김석윤   
100         100  101                   역린            이재규   
101         101  102        매드 맥스: 분노의 도로          조지 밀러   

                     provider        date country  n_scrn         money  \
0                   씨제이이앤엠(주)  2014-07-30      한국    1587  1.357480e+11   
1               롯데쇼핑㈜롯데엔터테인먼트  2017-12-20      한국    1912  1.157000e+11   
2                   씨제이이앤엠(주)  2014-12-17      한국     966  

In [168]:
name='국제시장'
df_cf_actor[df_cf_actor['영화명']==name]['배우'].values

array(['황정민,김윤진,오달수,정진영,장영남,라미란,김슬기,정영기,맹세창,김동원,김현중,김지영,황창기,박재우,박찬희,김예빈,김재철,김하연,김재현,이소영,류경현,황인준,이종석,이진희,전희경,박재언,유재상,김선영,정윤호,박길수,박영수,김근영'],
      dtype=object)

In [178]:
df_cf_actor[df_cf_actor['영화명']==name]['배우'].values

IndexError: index 0 is out of bounds for axis 0 with size 0

In [211]:
a=['톰 크루즈,소피아 부텔라,애나벨 월리스,러셀 크로우,하비에르 보텟,제이크 존슨,코트니 B. 반스,딜란 스미스']

a[0].split(',')[0]

'톰 크루즈'

In [235]:
len(df_cf_actor[df_cf_actor['영화명']=='쿵푸팬더 2']['배우'])

0

In [364]:
list=['1','1']

In [371]:
str(list).strip('[,]').replace('\'','').replace(', ','|')

'1|1'

In [399]:
df_realmovie.to_excel('movie_with_rat_act.xlsx',sheet_name='Sheet1')